# **Environment setup**

In [ ]:
"""This code mounts Google Drive to Colab"""
from google.colab import drive

# Mount Google Drive to the '/content/drive' directory
# and force remount if it is already mounted
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
"""This code changes the directory to Google Drive"""
import os
from pathlib import Path

# Set the base path to the Google Drive directory
base_path = "/content/drive/MyDrive"

# Change the current working directory to the Google Drive directory
os.chdir(f'{base_path}')

# Change directory to cloned repository
folder_name = "roof_recognition"
Path(f'{base_path}/{folder_name}').mkdir(parents=True, exist_ok=True)
os.chdir(f'{base_path}/{folder_name}')

# Print working directory
!pwd

# List files in directory
!ls

/content/drive/MyDrive/roof_recognition
config.json	  logs				      results
data		  model_20230209-194541_100_0.907.tf  runs
data.zip	  model-checkpts		      utils
LICENSE		  README.md			      yolov8n-seg.pt
loaded_model.zip  requirements.txt


In [ ]:
"""This code sets an environment variable"""

# Set the environment variable 'HYDRA_FULL_ERROR' to 1
%env HYDRA_FULL_ERROR=1

# Install required packages
!pip install ultralytics==8.0.0

env: HYDRA_FULL_ERROR=1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.8/219.8 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 19.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.3 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=78ed25985de76b4fbdc17a306b63cbfbe88df8ba686381addb1d66d4df698fc2
  Stored in directory: /root/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c4

In [ ]:
from ultralytics import YOLO

from PIL import Image
import numpy as np
import cv2
import tensorflow as tf
from osgeo import gdal
import matplotlib.pyplot as plt
import os
import yaml

In [ ]:
# checking availability of GPU resources
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# **Data Loading and Data Structure**

### Data Loading

In [ ]:
"""This code downloads data from Dropbox"""

# Download the data from Dropbox
# and save it as 'yolo-seg.zip'
!wget https://www.dropbox.com/s/a62mjk53n17whg9/yolo-seg.zip?dl=0 -O yolo-seg.zip

"""This code downloads model from Dropbox"""

# Download the data from Dropbox
# and save it as 'model.zip'
!wget https://www.dropbox.com/s/1s469jsvydvut7e/last.zip?dl=0 -O model.zip


--2023-02-22 03:56:27--  https://www.dropbox.com/s/a62mjk53n17whg9/yolo-seg.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/a62mjk53n17whg9/yolo-seg.zip [following]
--2023-02-22 03:56:28--  https://www.dropbox.com/s/raw/a62mjk53n17whg9/yolo-seg.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc08f3efe931cc18db798be8000d.dl.dropboxusercontent.com/cd/0/inline/B2-U5H-fHst0ULHmkgPl83mLxBGgDlk7-Xq-MptfqotLrOY-K19zBWFHL8pLui_kN8zC1mrLwOMXRm4TNdeWizd3G9PULRGM34HgsLCLbwFR6NFvXFkYBjgUfu9xeIzwxXCD17AhtfsmxZCEE6oe1dJCC-_XPR8KrCEXi8A-wM8YCQ/file# [following]
--2023-02-22 03:56:28--  https://uc08f3efe931cc18db798be8000d.dl.dropboxusercontent.com/cd/0/inline/B2-U5H-fHst0ULHmkgPl83mLxBGgDlk7-Xq-MptfqotLrOY-K19zBWFHL8pLui_kN8zC1mrLwOM

In [ ]:
"""This code extracts data from a zip file"""

# Set the path to the zip file
zipfilepath = "yolo-seg.zip"

# Set the target directory
target_dir = "./"

# Import the ZipFile class
from zipfile import ZipFile

# Extract the zip file to the target directory
with ZipFile(str(zipfilepath), 'r') as zipObj:
   zipObj.extractall(path=str(target_dir))

# Set the path to the zip file
zipfilepath = "model.zip"

# Set the target directory
target_dir = "./"

# Extract the zip file to the target directory
with ZipFile(str(zipfilepath), 'r') as zipObj:
   zipObj.extractall(path=str(target_dir))

### Data Structure

- yolo-seg
  - train
    - images
      - 000c57e6_74f7_4d78_ae65_5b7a6c50ce68.jpg
      - 003280f5_57ac_4e31_9011_ff081dce6208.jpg
    - labels
      - 000c57e6_74f7_4d78_ae65_5b7a6c50ce68.txt
      - 003280f5_57ac_4e31_9011_ff081dce6208.txt
  - valid
    - images
      - 003280f5_57ac_4e31_9011_ff081dce6208.jpg
      - 00a1af5d_7dbf_4c9e_81e1_d41d6f9aa7a8.jpg
    - labels
      - 003280f5_57ac_4e31_9011_ff081dce6208.txt
      - 00a1af5d_7dbf_4c9e_81e1_d41d6f9aa7a8.txt
  - classes.txt
  - ghana-seg.yaml

In [ ]:
ls ./yolo-seg

classes.txt  ghana-seg.yaml  test/  train/  valid/


#### **Classes**:
**Example of Schema**:

0 roof

1 car

2 human

3 animal

4 road

**Case with roofs**:

0 roof

In [ ]:
with open('./yolo-seg/classes.txt') as f:
    classes = f.readlines()
print("classes.txt:")
display(classes)


classes.txt:


['roof']

#### Yaml Configuration File

**path**: ./yolo-seg  # dataset root dir

**train**: train  # train images (relative to 'path') 946 images

**val**: test  # val images (relative to 'path') 165 images

**test**: test # test images (optional) 219 images

###### # classes
**names**:
  * 0: roof


In [ ]:
print("\nyaml file parameters:\n")
with open("./yolo-seg/ghana-seg.yaml", 'r') as stream:
    data_loaded = yaml.safe_load(stream)
display(data_loaded)



yaml file parameters:



{'path': './yolo-seg',
 'train': 'train',
 'val': 'valid',
 'test': 'test',
 'names': {0: 'roof'}}

#### **Label file**

* class_name x1_norm y1_norm x2_norm y2_norm x3_norm y3_norm
* class_name x1_norm y1_norm x2_norm y2_norm x3_norm y3_norm x4_norm y4_norm

# **Example with roofs**
* 0 0.07 0.17 0.06 0.23 0.0 0.22 0.0 0.16 0.07 0.17

# **Coordinates Normalization**
* X coordinates are normalized by the width of an image
* Y coordinates are normalized by the height of an image

In [ ]:
with open('./yolo-seg/train/labels/000c57e6_74f7_4d78_ae65_5b7a6c50ce68.txt') as f:
    labels_examples = f.readlines()
print("labels example:\n")
display(labels_examples)


labels example:



['0 0.99609375 0.35546875 0.99609375 0.578125 0.99609375 0.58984375 0.99609375 0.58984375 0.99609375 0.66015625 0.92578125 0.6484375 0.97265625 0.3515625 0.99609375 0.35546875\n',
 '0 0.109375 0.359375 0.12890625 0.48046875 0.140625 0.5703125 0.0 0.59375 0.0 0.375 0.109375 0.359375\n',
 '0 0.73046875 0.40625 0.70703125 0.5546875 0.5703125 0.53125 0.59375 0.38671875 0.73046875 0.40625\n',
 '0 0.95703125 0.39453125 0.9375 0.55859375 0.7265625 0.53125 0.74609375 0.3671875 0.95703125 0.39453125\n',
 '0 0.5859375 0.31640625 0.578125 0.375 0.55078125 0.37109375 0.546875 0.3984375 0.37890625 0.37890625 0.390625 0.2890625 0.5859375 0.31640625\n',
 '0 0.3125 0.26171875 0.3203125 0.3046875 0.3359375 0.3046875 0.33984375 0.359375 0.16015625 0.37890625 0.1484375 0.26171875 0.3125 0.26171875\n',
 '0 0.99609375 0.27734375 0.99609375 0.34765625 0.9765625 0.34765625 0.98828125 0.27734375 0.99609375 0.27734375\n',
 '0 0.9375 0.265625 0.921875 0.36328125 0.74609375 0.33984375 0.7578125 0.2421875 0.9375 

# YOLOv8 training

### YOLOv8 main hyperparams

* https://docs.ultralytics.com/
* https://github.com/ultralytics/ultralytics
* https://github.com/ultralytics/ultralytics/blob/main/ultralytics/yolo/cfg/default.yaml


In [ ]:
model_name = 'yolov8n-seg.pt' # path to model file, i.e. yolov8n-seg.pt, yolov8n-seg.yaml
data = './yolo-seg/ghana-seg.yaml' # path to data file, i.e. i.e. coco128.yaml
epochs = 3 # number of epochs to train for
resume = False  # resume training from last checkpoint
imgsz = 256 # size of input images as integer or w,h
optimizer = "Adam" # optimizer to use, choices=['SGD', 'Adam', 'AdamW', 'RMSProp']
lr0 = 3E-04  # initial learning rate (i.e. SGD=1E-2, Adam=1E-3)


### Model training

In [ ]:
model = YOLO(model_name)  # load a pretrained YOLOv8n-seg segmentation model
results = model.train(data=data, epochs=epochs, resume=resume, imgsz=imgsz, optimizer=optimizer, lr0=lr0)  # train the model

yolo/engine/trainer: task=segment, mode=train, model=yolov8n-seg.yaml, data=./yolo-seg/ghana-seg.yaml, epochs=3, patience=50, batch=16, imgsz=256, save=True, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, retina_masks=False, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, workspace=4, nms=False, lr0=0.0003, lrf=0.01, momentum=0.937, weight_decay=0.001, warmup_epochs=

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

# Loading pretrained model and predicting results

In [ ]:
model_name = "./last.pt"
model = YOLO(model_name)

### Predicting examples

In [ ]:
from osgeo import gdal
import matplotlib.pyplot as plt


def plot_result_yolo(image_id, pred_mask, folder_mask, folder_chips, class_names):

  f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(30, 15))

  dataset = gdal.Open(f'{base_path}/{folder_name}/{folder_chips}/{image_id.replace(".mask", "")}')
  image = np.dstack([dataset.GetRasterBand(i+1).ReadAsArray()
                     for i in range(dataset.RasterCount)])
  ax1.imshow(image, alpha=1)

  dataset = gdal.Open(f'{base_path}/{folder_name}/{folder_mask}/{image_id}')
  true_mask = dataset.GetRasterBand(1).ReadAsArray()
  ax2.imshow(true_mask, cmap="magma")
  ax2.imshow(image, alpha=0.7)

  ax3.imshow(pred_mask, cmap="magma")
  ax3.imshow(image, alpha=0.7)

  ax1.axis('off')
  ax2.axis('off')
  ax3.axis('off')
  plt.show()

  score = compute_metrics(true_mask, pred_mask, class_names)
  return score


In [ ]:
def compute_metrics(y_true, y_pred, class_names):
  '''
  Computes IOU and Dice Score.

  Args:
    y_true (tensor) - ground truth label map
    y_pred (tensor) - predicted label map
  '''
  
  class_wise_iou = []
  class_wise_dice_score = []

  smoothening_factor = 0.00001

  for i in range(len(class_names)):
    intersection = np.sum((y_pred == i) * (y_true == i))
    y_true_area = np.sum((y_true == i))
    y_pred_area = np.sum((y_pred == i))
    combined_area = y_true_area + y_pred_area
    
    iou = (intersection + smoothening_factor) / (combined_area - intersection + smoothening_factor)
    class_wise_iou.append(iou)
    
    dice_score =  2 * ((intersection + smoothening_factor) / (combined_area + smoothening_factor))
    class_wise_dice_score.append(dice_score)

  iou_df = pd.DataFrame({class_name: class_wise_iou[i] * 100 for i, class_name in enumerate(class_names)}, index=["IoU Score"])
  dice_df = pd.DataFrame({class_name: class_wise_dice_score[i] * 100 for i, class_name in enumerate(class_names)}, index=["Dice Score"])
  scores = pd.concat([iou_df, dice_df], axis=0)
  scores["average_score"] = scores.iloc[:,:3].sum(axis=1)/3

  return scores

In [ ]:
import os
import pandas as pd

scores = pd.DataFrame()
class_names = ["background", "buildings"]
folder_mask = "data/test-multimasks"
folder_chips = "data/test-chips"
images = os.listdir(f"{base_path}/{folder_name}/{folder_mask}")

for image_id in images[:]:

  input = f"{base_path}/{folder_name}/{folder_chips}/{image_id.replace('.mask','')}"

  results = model(source=input, conf=0.3)
  masks = ((results[0][1].sum(axis=0)>0)*1).cpu().numpy()

  score = plot_result_yolo(image_id, masks, folder_mask, folder_chips, class_names)
  scores = scores.append(score, ignore_index=True, sort=False)

  display(score)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("IoU Score")
display(scores.iloc[::2].describe())

print("Dice Score")
display(scores.iloc[1::2].describe())

IoU Score


,background,buildings,average_score
count,24.000000,24.000000,24.000000
mean,61.642924,63.146528,41.596484
std,12.175981,8.652054,3.388896
min,39.216654,38.148749,36.436092
25%,54.397675,60.620951,39.193650
50%,63.353836,63.845422,42.096298
75%,68.100560,68.732223,43.076684
max,88.314435,74.375400,50.350723


Dice Score


,background,buildings,average_score
count,24.000000,24.000000,24.000000
mean,75.587999,77.053053,50.880351
std,9.495270,7.058700,2.713463
min,56.339027,55.228512,46.680032
25%,70.461980,75.482810,48.532948
50%,77.564250,77.933708,51.580157
75%,81.020260,81.464215,52.153393
max,93.794652,85.304923,56.965842


# Additional Code

In [ ]:
# !yolo task=detect mode=predict model=yolov8n.pt source="/content/drive/MyDrive/roof_recognition/people.png"

In [ ]:
# !yolo task=detect mode=predict model=yolov8n.pt source="/content/drive/MyDrive/roof_recognition/video.mp4"

In [ ]:
# !yolo task=segment mode=predict model="runs/segment/train17/weights/best.pt" \
# source="/content/drive/MyDrive/roof_recognition/yolo-seg/test/images" conf=0.5

In [ ]:
# model.val('./yolo-seg/ghana-seg.yaml')